# Aprendizagem de Máquina I

## Hugo Tremonte de Carvalho

#### hugo@dme.ufrj.br

O [Breast Cancer Wisconsin Dataset](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic) é um conjunto de dados real, amplamente usado em aprendizado de máquina para tarefas de classificação binária. Ele contém informações obtidas a partir de imagens digitais de biópsias de tumores mamários, com o objetivo de distinguir entre tumores malignos e benignos.

Cada amostra representa o resultado da análise de núcleos de células presentes na imagem, de onde foram extraídas 30 características numéricas relacionadas à forma, textura e estrutura dos núcleos. O conjunto inclui 569 amostras, sendo aproximadamente 62% benignas e 38% malignas.

As covariáveis são estatísticas-resumo de:
* `radius` (distâncias do cento até o perímetro)
* `texture` (tons de cinza da imagem)
* `perimeter`
* `area`
* `smoothness` (variacão local de raio)
* `compactness` (perímetro²/área - 1.0)
* `concavity` (grau de concavidade dos contornos)
* `concave points` (número de porçoes côncavas no contorno)
* `symmetry`
* `fractal dimension` ("coastline approximation" - 1)

_The mean, standard error, and "worst" or largest (mean of the three worst/largest values) of these features were computed for each image, resulting in 30 features._

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

from sklearn.datasets import load_breast_cancer

from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import RocCurveDisplay, ConfusionMatrixDisplay, confusion_matrix, roc_curve

import numpy as np

import seaborn as sns

In [ ]:
########## PRETTY MATRIX DISPLAY ##########
# Source: https://gist.github.com/braingineer/d801735dac07ff3ac4d746e1f218ab75
def matprint(mat, fmt="g", round_dig = 0):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            if round_dig == 0:
                print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
            else:
                print(("{:"+str(col_maxes[i])+fmt+"}").format(round(y, round_dig)), end="  ")
        print("")
###########################################

########## CONFUSION MATRIX METRICS ##########
class ConfusionMatrixMetrics:
    def __init__(self, matrix):
        """
        Inicializa a classe com uma matriz de confusão 2x2.
        A matriz deve estar no formato:
        [[VN, FP],
         [FN, VP]]
        onde:
        - VP: Verdadeiros Positivos
        - FP: Falsos Positivos
        - FN: Falsos Negativos
        - VN: Verdadeiros Negativos
        """
        self.matrix = matrix

        self.VN = self.matrix[0][0]
        self.FP = self.matrix[0][1]
        self.FN = self.matrix[1][0]
        self.VP = self.matrix[1][1]

        self.N = self.matrix[0][0] + self.matrix[0][1]
        self.P = self.matrix[1][0] + self.matrix[1][1]

        self.Pop = self.N + self.P

    def prev(self):
        """Calcula a prevalência da classe positiva"""
        try:
            return (self.P)/(self.Pop)
        except ZeroDivisionError:
            return 0.0

    def acc(self):
        """Calcula a acurária"""
        try:
            return (self.VN + self.VP)/self.Pop
        except ZeroDivisionError:
            return 0.0

    def FPR(self):
        """Calcula a FPR"""
        try:
            return self.FP/self.N
        except ZeroDivisionError:
            return 0.0

    def TNR(self):
        """Calcula a TNR"""
        try:
            return self.VN/self.N
        except ZeroDivisionError:
            return 0.0

    def TPR(self):
        """Calcula a TPR"""
        try:
            return self.VP/self.P
        except ZeroDivisionError:
            return 0.0

    def FNR(self):
        """Calcula a FNR"""
        try:
            return self.FN/self.P
        except ZeroDivisionError:
            return 0.0

    def FOR(self):
        """Calcula a FOR"""
        try:
            return self.FN/(self.VN + self.FN)
        except ZeroDivisionError:
            return 0.0

    def PPV(self):
        """Calcula o PPV"""
        try:
            return self.VP/(self.FP + self.VP)
        except ZeroDivisionError:
            return 0.0

    def NPV(self):
        """Calcula a NPV"""
        try:
            return self.VN/(self.VN + self.FN)
        except ZeroDivisionError:
            return 0.0

    def FDR(self):
        """Calcula a FDR"""
        try:
            return self.FP/(self.FP + self.VP)
        except ZeroDivisionError:
            return 0.0

    def F1(self):
        """Calcula a F1"""
        try:
            return 2/(1/self.PPV() + 1/self.TPR())
        except ZeroDivisionError:
            return np.NaN

    def print(self):
        """Mostra todas as métricas calculadas acima"""
        print('Matriz de confusão:')
        matprint(self.matrix)

        print('\n')

        print('Prevalência:', np.round(self.prev(), 3))
        print('Acurácia:', np.round(self.acc(), 3))

        print('\n')

        print('Taxa de falsos positivos:', np.round(self.FPR(), 3))
        print('Taxa de verdadeiros negativos (Especificidade):',np.round(self.TNR(), 3))
        print('Taxa de verdadeiros positivos (Recall):', np.round(self.TPR(), 3))
        print('Taxa de falsos negativos:', np.round(self.FNR(), 3))

        print('\n')

        print('False omission rate:', np.round(self.FOR(), 3))
        print('Valor preditivo positivo (Precisão):', np.round(self.PPV(), 3))
        print('Valor preditivo negativo:', np.round(self.NPV(), 3))
        print('False discovery rate:', np.round(self.FDR(), 3))

        print('\n')

        print('F1 Score:', np.round(self.F1(), 3))
    ##############################################

a) Carregue a base de dados e faça uma análise exploratória.

In [ ]:
data = load_breast_cancer(as_frame = True)

In [ ]:
data

In [ ]:
data = data.frame

In [ ]:
data.shape

In [ ]:
data.isna().sum().sum()

In [ ]:
data.head()

In [ ]:
sns.pairplot(data = data, hue = 'target')

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data['target'].value_counts()

# https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html
# 0 = maligno
# 1 = benigno

b) Treine e compare o desempenho do SVM, com kernel linear

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns = ['target']), data['target'], test_size=0.3)

In [ ]:
SVC_linear_pipe = Pipeline([('Scaler', StandardScaler()), ('SVC', SVC(kernel = 'linear'))])

# COMPARAR COM https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html

param_grid_linear = {
    'SVC__C': np.logspace(-1, 2, 30),
}

SVC_pipe_linear_CV = GridSearchCV(SVC_linear_pipe, param_grid = param_grid_linear, cv=5, scoring='accuracy', n_jobs=-1)
SVC_pipe_linear_CV.fit(X_train, y_train)

In [ ]:
np.logspace(-1, 2, 30)

In [ ]:
CM_SVC_linear = confusion_matrix(y_test, SVC_pipe_linear_CV.predict(X_test))

ConfusionMatrixMetrics(CM_SVC_linear).print()

c) Verifique o desempenho do SVM com outros kernel (polinomial e rbf)

d) Verifique o desempenho dos modelos baseados em árvores e do KNN